In [ ]:
from ipyleaflet import Map, basemaps, GeoJSON, WidgetControl
from ipywidgets import Text, HTML
import json
import pandas as pd

In the beginning, we load geojson data of taxi zones in NY.

In [ ]:
with open('../resources/NYC_Taxi_Zones.geojson', 'r') as f:
    taxi_zones_data = json.load(f)

The next step is loading processed clean data of taxis in NY. Then, the number of pickups is counted and divided into equal bins for each zone.

In [ ]:
pickup_locations = pd.read_csv('../processed_data/2018-taxi-trip-data-clean.csv', usecols = ['pickup_location'])
pickup_locations_binned = pd.cut(pickup_locations.pickup_location.value_counts(), bins = 50, labels = range(49, -1, -1))

Creating useful functions to visualize the map:

- get_color - the function returns the proper color, that corresponds with the number of pickups, for the given zone.
- update_html - the function adds the information label to the map.

In [ ]:
def get_color(feature):
    location_id = int(feature['properties']['location_id'])
    color_index = 49
    
    try: 
        color_index = pickup_locations_binned[location_id]
    except KeyError:
        pass
    
    colors = ['#660F00','#6C1000','#721100','#781200','#7F1300','#851400','#8B1500',
              '#911600','#971700','#9D1700','#A31800','#A91900','#AF1A00','#B61B00',
              '#BC1C00','#C21D00','#C81E00','#CE1F00','#D42000','#DA2100','#E02100',
              '#E72200','#ED2300','#F32400','#F92500','#FF2600','#FF3108','#FF3C11',
              '#FF4719','#FF5121','#FF5B29','#FF6532','#FF6E3A','#FF7742','#FF804A',
              '#FF8953','#FF915B','#FF9A63','#FFA16B','#FFA974','#FFB07C','#FFB784',
              '#FFBE8D','#FFC495','#FFCB9D','#FFD1A5','#FFD6AE','#FFDCB6','#FFE1BE',
              '#FFE5C6','#FFEACF'] 
    
    return {'fillColor': colors[color_index]}

def update_html(feature, **kwargs):
    location_id = int(feature['properties']['location_id'])
    pickup_count = 0
    
    try: 
        pickup_count = pickup_locations.pickup_location.value_counts()[location_id]
    except KeyError:
        pass
    
    html.value = """
        <h3><b>{}</b></h3>
        <h4>Pickup count: {}</h4>
    """.format(
        feature['properties']['zone'],
        pickup_count
    )

Once we have both the needed data and the useful functions we create the map that displays the result.

In [ ]:
center = [40.6970, -74.0121]
zoom = 9.5

ny_map = Map(basemaps = basemaps.OpenStreetMap.Mapnik, center = center, zoom = zoom)

taxi_zones_geojson = GeoJSON(
    data = taxi_zones_data,
    style  = {
        'color': '#555555', 'opacity': 1, 'fillOpacity': 0.8, 'weight': 0.6
    },
    style_callback = get_color
)
taxi_zones_geojson.on_hover(update_html)

html = HTML("""Hover over a zone""")
html.layout.margin = "0px 20px 0px 20px"
text_box = WidgetControl(widget = html, position="topright")

ny_map.add_layer(taxi_zones_geojson)
ny_map.add_control(text_box)

The final step is to display the map.

In [ ]:
display(ny_map)